# Calibrate solar and volcanic


We have to use HadGEM3-GC31-LL RFMIP experiments, as UKESM didn't do them. But since the radiation code is the same, and I don't think the chemistry changes in UKESM1 is going to affect anything, I think this is OK.

Equations to use:
- volcanic = -19.742 AOD + 0.31227
- solar = 0.168197 TSI - 228.097

In [ ]:
import pooch
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
from scipy.stats import linregress

In [ ]:
solar_forcing_file = pooch.retrieve(
    url="http://aims3.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/CMIP/SOLARIS-HEPPA/SOLARIS-HEPPA-3-2/atmos/mon/multiple/gm/v20170103/solarforcing-ref-mon_input4MIPs_solar_CMIP_SOLARIS-HEPPA-3-2_gn_185001-229912.nc",
    known_hash="md5:e12977b6b1ce2e883e26ac3fd2139391",
)

In [ ]:
nc = Dataset(solar_forcing_file)
tsi_monthly = nc.variables['tsi'][:]
nc.close()

In [ ]:
years = np.arange(1850, 2300, dtype=int)
isleap = np.zeros(450)
isleap[np.logical_and(years%4==0, np.logical_or(years%100!=0, years%400==0))] = 1

In [ ]:
tsi = np.zeros(450)
for i, year in enumerate(years):
    weights = [31,28+isleap[i],31,30,31,30,31,31,30,31,30,31]
    tsi[i] = np.average(tsi_monthly[(i)*12:(1+i)*12], weights=weights)

In [ ]:
tsi_baseline = np.mean(tsi[:24])
tsi_baseline

In [ ]:
pl.plot(np.arange(1850, 2300), tsi)
pl.axhline(tsi_baseline, ls=':', color='k')

In [ ]:
rfmip_tier2 = pd.read_csv('../data/smith2021/RFMIP-ERF-tier2.csv', index_col=0)
natural = rfmip_tier2[['HadGEM3-GC31-LL NAT']]
natural

In [ ]:
volcanic_saod = pooch.retrieve(
    'ftp://iacftp.ethz.ch/pub_read/luo/CMIP6/CMIP_1850_2014_extinction_550nm_strat_only_v3.nc',
    known_hash = 'md5:ab97e78faf5fdccdf67cfd7ced68b20e'
)

In [ ]:
nc = Dataset(volcanic_saod)
ext = nc.variables['ext550'][:].transpose((2,1,0))  # time, height, lat
lev = nc.variables['altitude'][:]
lat = nc.variables['latitude'][:]
time = nc.variables['month'][:]
print(nc.variables['month'])
nc.close()

In [ ]:
lat_bnds = np.concatenate(([-90], 0.5*(lat[1:]+lat[:-1]), [90]))
weights = np.diff(np.sin(np.radians(lat_bnds)))
tax = np.zeros(165*12)
aod_cmip6 = np.zeros(165*12)
for i in range(0,1970,12):
    gl_mn_OD = np.average(np.sum(np.mean(ext[i:i+12,...],axis=0) * 0.5 ,axis=0),weights=weights) # 0.5 is thickness in km

for i in range(1980):
    aod_cmip6[i] = np.average(np.sum(ext[i,...] * 0.5,axis=0),weights=weights)

In [ ]:
aod_cmip6

In [ ]:
gl_mn_OD

In [ ]:
years = np.arange(1850, 2015, dtype=int)
aod = np.zeros(165)
for i, year in enumerate(years):
    weights = [31,28+isleap[i],31,30,31,30,31,31,30,31,30,31]
    aod[i] = np.average(aod_cmip6[(i)*12:(1+i)*12], weights=weights)

In [ ]:
pl.plot(aod)

In [ ]:
pl.scatter(aod, natural.values[:165])
sl, ic, _,_,_ = linregress(aod, natural.values[:165].squeeze())
pl.plot(np.linspace(0,0.12,100), sl*np.linspace(0,0.12,100)+ic)
sl, ic

In [ ]:
volcanic_1850_2300 = np.zeros((451))
volcanic_1850_2300[1:166] = sl * aod + ic
volcanic_1850_2300[165:175] = np.linspace(volcanic_1850_2300[165], 0, 10)

In [ ]:
resid = natural.values[:165].squeeze() - (sl * aod + ic)
pl.plot(resid)

In [ ]:
pl.scatter(tsi[:165], resid)
sl, ic, _,_,_ = linregress(tsi[:165], resid)
pl.plot(np.linspace(1360.4,1362,100), sl*np.linspace(1360.4,1362,100)+ic)
sl, ic

In [ ]:
tsi.shape

In [ ]:
solar_1850_2300 = np.zeros(451)
solar_1850_2300[1:] = sl * tsi + ic

In [ ]:
pl.plot(np.arange(1850, 2301), volcanic_1850_2300)

In [ ]:
pl.plot(np.arange(1850, 2301), solar_1850_2300)

In [ ]:
pl.plot(solar_1850_2300 + volcanic_1850_2300)

In [ ]:
df_out = pd.DataFrame({
    'solar': solar_1850_2300,
    'volcanic': volcanic_1850_2300
}, index = np.arange(1850, 2301))

In [ ]:
df_out.loc[1993]  # time bound, takes the weight of forcing across 1992 year

In [ ]:
df_out.to_csv('../data/calibration/natural_forcing.csv')